In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
sys.executable

'c:\\Users\\denis\\anaconda3\\envs\\ucm\\python.exe'

# Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
from src.scrapper import scrape_nba_finals, scrape_NBA_team_data

# Get data

**Example NBA finals**

In [4]:
scrape_nba_finals()

,Year,Lg,Champion,Runner-Up,Finals MVP,,Points,Rebounds,Assists,Win Shares
0,2023,NBA,Denver Nuggets,Miami Heat,N. Jokić,,N. Jokić (600),N. Jokić (269),N. Jokić (190),N. Jokić (5.0)
1,2022,NBA,Golden State Warriors,Boston Celtics,S. Curry,,J. Tatum (615),A. Horford (214),J. Tatum (148),J. Butler (3.8)
2,2021,NBA,Milwaukee Bucks,Phoenix Suns,G. Antetokounmpo,,G. Antetokounmpo (634),G. Antetokounmpo (269),J. Holiday (199),G. Antetokounmpo (3.7)
3,2020,NBA,Los Angeles Lakers,Miami Heat,L. James,,A. Davis (582),L. James (226),L. James (184),A. Davis (4.5)
4,2019,NBA,Toronto Raptors,Golden State Warriors,K. Leonard,,K. Leonard (732),D. Green (223),D. Green (187),K. Leonard (4.9)
5,2018,NBA,Golden State Warriors,Cleveland Cavaliers,K. Durant,,L. James (748),D. Green (222),L. James (198),L. James (5.2)
6,2017,NBA,Golden State Warriors,Cleveland Cavaliers,K. Durant,,L. James (591),K. Love (191),L. James (141),L. James (4.3)
7,2016,NBA,Cleveland Cavaliers,Golden State Warriors,L. James,,K. Thompson (582),D. Green (228),R. Westbrook (198),L. James (4.7)
8,2015,NBA,Golden State Warriors,Cleveland Cavaliers,A. Iguodala,,L. James (601),D. Howard (238),L. James (169),S. Curry (3.9)
9,2014,NBA,San Antonio Spurs,Miami Heat,K. Leonard,,K. Durant (563),T. Duncan (211),R. Westbrook (153),L. James (4.3)


**Example team season**

In [4]:
scrape_NBA_team_data()

no division: Atlantic Division
no division: Central Division
no division: Southeast Division
no division: Northwest Division
no division: Pacific Division
no division: Southwest Division
no division: Midwest Division


AttributeError: 'DataFrame' object has no attribute 'append'